Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE`/`raise NotImplementedError` or "YOUR ANSWER HERE", as well as your name and collaborators below:

# 05_HW2: More about getting data over the web

## Objectives

- Solidifying knowledge of web-scraping
  - Practice with Programmatic manipulation of tree structured data
  - Seeing similarities and differences between JSON and XML
  - Deeper understanding of the XML format and tree
  - Relating back to tabular data structres by processing into DataFrame
- Practice with APIs, keys, GET, and POST

Recall the extended `buildURL` from the previous homework (which has a parameter `extension` for the format of the resource you are seeking), provided below for your convenience.

In [ ]:
from IPython.core.debugger import set_trace
import requests
from lxml import etree
import pandas as pd
import re
import io
import json

def buildURL(location, resource, protocol='http', extension=None):
    fmt1 = '{}://{}{}'
    if resource[0] != '/':
        resource = '/' + resource
    url = fmt1.format(protocol, location, resource)
    if extension:
        url += '.' + extension
    return url



**Q1** Please follow the model above (and your previous HW) to generalize the `HTTPGetParams` function to allow optional arguments to specify a protocol and an extension, and use the revised `buildURL` function to help do its work.  Further, let the `pdict` parameter dictionary be optional, so that, if not specified, the request is made without URL parameters. 

    HTTPGet(location, resource, protocol='http', extension=None, pdict=None)

Finally, the function should extend what it does *with* the result so that, if the returned content is JSON, we, like before, interpret the JSON format into a Python data structure, but if the content is XML, we interpret the XML format into an `Element` data structure (Note, do not yield an `ElementTree`). and if the content is HTML, we also interpret into an `lxml` `Element` data structure. Do not use the `extension` parameter to make this determination, but rather use the Content type header field available in the response.  Think about flexibility when looking for matching a content-type.  Some providers may or may not include information about encoding in the content-type.  Others may say `text/json` as opposed to `application/json`.

If the content is neither json nor xml nor html, but the request was successful, we should return the string of the byte-stream body.  If the request was unsuccessful, we should return `None`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Testing cell

d = HTTPGet('covidtracking.com','/api/states/daily','https','json',{'state':'NY', 'date':20200316})
assert type(d) is dict
assert d['state'] == 'NY'
assert 'posNeg' in d
assert d['date'] == 20200316

In [ ]:
# Testing cell

protocol = "http"
location = "personal.denison.edu"
resource = "/~bressoud/datasystems/data/mystery5"
x = HTTPGet(location, resource)
assert x[:10] == b'\xff\xd8\xff\xe0\x00\x10JFIF'

In [ ]:
# Testing cell

d = HTTPGet('en.wikipedia.org','/wiki/List_of_most_popular_given_names_by_state_in_the_United_States','https')
assert(len(d) == 2)
assert(type(d) is etree._Element)

In [ ]:
# Testing cell

data = HTTPGet('api.kivaws.org', '/v1/loans/newest', protocol='https', 
                   extension='xml', pdict = {'status': 'fundraising'})
assert (type(data) is etree._Element)
assert len(data) == 2

## Provider API Function

We often find that a provider has grouped functionality logically into a set of endpoints, each able to obtain data that corresponds to a single logical subset of the data.  In our Python programming, we should refelect this by providing one or a small number of functions that abstract obtaining data from this endpoint.  As one example, **Q4** does this for the Kiva newest loans endpoint.

**Q2** We know that the Kiva endpoint for the most recent loans uses the location of `'api.kivaws.org'` and the base resource is `'/v1/loans/newest'`. Please look at the inclass worksheet for more details about path endpoints and query parameters for this resource. It provides examples like

http://api.kivaws.org/v1/loans/newest.xml?status=fundraising

http://api.kivaws.org/v1/loans/newest.json?status=fundraising

Note from the provider's documentation that they support formats of html, json, and xml, and from the examples note that we suffix the resource with a period and the desired extension.  Our goal in this question is to build a function that retrieves the newest loans and allows the caller to specify the page number,the loans per page, and the format.  So write the function

`
kivaNewestLoans(pagenum=1, loansperpage=20, datatype='json')
`

that uses your previous `HTTPGet` function and realizes this nice general Python function for obtaining the Kiva newest loans. It should work for any of the three data types mentioned, but should default to `json`. Note the default parameters to allow us to call `kivaNewestLoans` with no arguments and get reasonable results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Testing cell
d = getKivaNewestLoans(5,30,'json')
assert type(d) is dict
assert(len(d) == 2)
assert 'paging' in d.keys()
assert 'loans' in d.keys()
assert len(d['paging']) == 4
assert len(d['loans']) == 30

d = getKivaNewestLoans()
assert type(d) is dict
assert(len(d) == 2)
assert 'paging' in d.keys()
assert 'loans' in d.keys()

d = getKivaNewestLoans(5,30,'xml')
assert type(d) is etree._Element
assert(len(d) == 2)

## Tree Data Structure to DataFrame

**Q3** Write a function

    kivaNewestLoansJsonDF(columns, pagenum=1, loansperpage=20)

that retrieves json data through the kiva newest loans endpoint, using the given `pagenum` and `loansperpage`, and then builds a `pandas` dataframe from the json result.  The columns parameter is a list of strings specifying the columns to be included in the result.  For a json-based data structure, these strings correspond to one of the **keys** in the dictionary that specifies each individual loan.  You may assume that, for any particular `columns` list entry, the child item is a *leaf* of the tree.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

df = kivaNewestLoansJsonDF(['name','sector','loan_amount'])
df.head()

In [ ]:
# Testing cell
cols = ['name','sector','loan_amount']
df = kivaNewestLoansJsonDF(cols)
assert df.shape == (20,3)

df = kivaNewestLoansJsonDF(['name','loan_amount'],2,35)
assert df.shape == (35,2)
assert 'name' in list(df.columns)
assert 'loan_amount' in list(df.columns)

**Q4** Write a function

    kivaNewestLoansXMLDF(columns, pagenum=1, loansperpage=20)

that retrieves xml data through the kiva newest loans endpoint, using the given pagenum and loansperpage, and then builds a pandas dataframe from the xml result.  The columns parameter is a list of strings specifying the columns to be included in the result.  For an xml-based data structure, these strings correspond to one of the **tags** that are children for each individual loan.  You may assume that, for any particular columns list entry, the child item is a *leaf* of the tree, so the text of the Element gives the desired value.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

df = kivaNewestLoansXMLDF(['name','sector','loan_amount'])
df.head()

In [ ]:
# Testing cell
cols = ['name','sector','loan_amount']
df = kivaNewestLoansXMLDF(cols)
assert df.shape == (20,3)

df = kivaNewestLoansXMLDF(['name','loan_amount'],2,35)
assert df.shape == (35,2)
assert 'name' in list(df.columns)
assert 'loan_amount' in list(df.columns)

## Practice with APIs and POST

We conclude this homework with practice very similar to the book. Using the TMDB API and the `requests` module, you will

	1.	Create a guest session
	2.	Find a TV show based on a given criteria
	3.	Rate the TV show with a given rating

As we saw in the reading, most APIs require some form of authentication from the client, especially for a POST request. We have selected exercises below that do not require OAuth (to be discussed in chapter 24), but that do require an `apikey` and a guest session. First, we introduce an even more general `buildURL` function (which also specifies the `port`). Please run the cell that follows.

In [ ]:
def buildURL(resource_path, host="http.bin", protocol="https", 
             extension=None, port=None):
    if resource_path[0] != '/':
        resource_path = '/' + resource_path
    
    if extension != None:
        resource_path += "." + extension
        
    if port != None:
        host = host + ":{}".format(port)
    
    url_template = "{}://{}{}"
    url = url_template.format(protocol, host, resource_path)
    return url

### JSON for `apikey`

We will use the same API discussed in the book, TMDB. Recall from earlier in the course that it is very unwise to hard code sensitive information into a program you are writing. For instance, by including a real `apikey` in the textbook, the authors make it possible for a reader to behave badly using this real key, and to cause trouble with TMDB.

For the exercise that follows, you will have to use your own api_key. Rather than hard-coding it into this notebook, please open `creds.json` and type in your `apikey` there (as a string). Then run the cell below.

In [ ]:
with open("./creds.json") as f:
    creds = json.load(f)
protocol = creds['tmdb']['protocol']
location = creds['tmdb']['location']
apikey = creds['tmdb']['apikey']
sessionid = creds['tmdb']['sessionid']

In [ ]:
# Testing cell


**Q5** Check the book or the TMDB API to find the correct resource path to request a new guest session ID. Please write a function `get_guest_session_id(my_api_key)` that uses the given API key to get and return a guest session id. For fun, think about how to modify your function so that instead of returning the guest session id, you would instead modify the JSON file to write the guest session id into it. Please use the `buildURL` function in your answer.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# We test the function using the apikey we got from the JSON you modified
print(get_guest_session_id(apikey))

In [ ]:
# Testing cell

assert len(get_guest_session_id(apikey)) == 32

In [ ]:
# Testing cell


**Q6** Study the TMDB API documentation to find the correct syntax for searching for a TV show. Then write a function `find_first_show(my_api_key, q, ad)` that searches for a TV show with query `q`, and with `ad` as the option for whether or not to include adult shows. Please return the `id` of the top result. Be careful: we do not want the genre_id. This will be much easier if you use the "Try it out" feature on the API documentation. See the test cells for a sample invocation.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# This uses the API key from the JSON file, if you ran all cells above
print(find_first_show(apikey,"Pretty Little Liars: The Perfectionists","false"))


In [ ]:
# Testing cell
assert find_first_show(apikey,"Pretty Little Liars: The Perfectionists","false") == 79863
assert find_first_show(apikey,"TURN: Washington's Spies","true") == 57276


**Q7** Study the TMDB API documentation to find the correct syntax for posting a rating of a TV show. Then write a function `post_tv_rating(my_api_key, tv_id, rat)`, that posts a rating of `rat` (a float) for the given `tv_id` (an integer), returning the `response` you get from the `requests` module. Your function should call `get_guest_session_id` to get a session id. Please don't forget the required header parameter(s).

In [ ]:
# Solution cell
# YOUR CODE HERE
raise NotImplementedError()

# This uses the API key from the JSON file
r = post_tv_rating(apikey,79863,7.8)
print(type(r))
print()
print(r.json())
print()
print(r.url)

In [ ]:
# Testing cell

r = post_tv_rating(apikey,79863,7.8)

assert r.json()['status_code'] == 1
assert r.json()['status_message'] == 'Success.'
assert '79863' in r.url
assert apikey in r.url
assert 'tv' in r.url

r = post_tv_rating(apikey,57276,5.2)
assert '57276' in r.url
